In [1]:
from Xray.pipeline.training_pipeline import TrainPipeline
from Xray.entity.artifact_entity import DataIngestionArtifact

artifact = DataIngestionArtifact(train_file_path=r"D:\Projects\deeplearningproject\artifact\11_05_2024_08_32_32\data_ingestion\data\train",
                                 test_file_path=r"D:\Projects\deeplearningproject\artifact\11_05_2024_08_32_32\data_ingestion\data\test")

pipe = TrainPipeline()



In [2]:
artifact.test_file_path

'D:\\Projects\\deeplearningproject\\artifact\\11_05_2024_08_32_32\\data_ingestion\\data\\test'

In [3]:
data_artifact = pipe.start_data_transformation(data_ingestion_artifact=artifact)

In [16]:
import os
os.path.exists(artifact.train_file_path)

True

In [4]:
data_artifact.test_transform_file_path

'artifact\\11_05_2024_08_55_10\\data_transformation\\test_transforms.pkl'